In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

### Load phrase base

In [2]:
import pickle

def load_phrase_base(file_name: str) -> (dict, list):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
        phrase_base = data.get('phrase_base', {})
        marks = data.get('marks', {})
        return phrase_base, marks
    
phrase_base, marks = load_phrase_base('./data/texts/phrase_base.pickle')
# phrase_base, marks = load_phrase_base('./data/texts/monotone_base.pickle')

### Draw notations

In [ ]:
from ipythonblocks import BlockGrid

def draw_notation(notation: np.array):
    bit_grid = BlockGrid(len(notation), 1, fill=(17, 41, 129))
    for block in range(bit_grid.width):
        color = bit_grid[0, block]
        if notation[block]:
            bit_grid[0, block] = (244, 195, 173)
    bit_grid.lines_on = False
    bit_grid.show()

def draw_note_notations(notes, note_idx: int):
    for i in range(len(notes[note_idx])):
        draw_notation(notes.note_notation_as_bits(note_idx, i))

In [ ]:
abc_notes = BitNotes(note_count=26, 
                     notation_count=5, 
                     active_bits=8, 
                     bit_count=255)

key = list(phrase_base.keys())[0]
phrase = phrase_base[key][0]
bit_chord = abc_notes.phrase_chord(phrase)
draw_notation(abc_notes.notation_as_bits(bit_chord))
print(phrase)
for note_idx, notation_idx in phrase:
    notation = abc_notes.note_notation_as_bits(note_idx, notation_idx)
    draw_notation(notation)

In [ ]:
feeder = PhraseFeeder(phrase_base, marks)    

In [ ]:
phrases, output_bits = feeder.take_phrase_bunch()
print(output_bits, marks[output_bits])
print(phrases)

### Learn model

In [3]:
def feed_phrase_batch(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, count=200):
    phrases, output_bits = feeder.take_phrase_batch(count=200, random_bit_key=False)
    mark = feeder.marks[output_bits]    
    output_bits = set(output_bits)
    for phrase in phrases:
        bit_chord = bit_notes.phrase_chord_as_bits(phrase)
        cxnn.receive_bits(input_bits=bit_chord, output_bits=output_bits)
    return mark    

def feed_n_batches(cxnn: ContextNN, feeder: PhraseFeeder, bit_notes: BitNotes, n=10, batch_size=200):
    for i in range(n):
        mark = feed_phrase_batch(cxnn, feeder, bit_notes, count=batch_size)
        print(f'batch {i+1}/{n} {mark}')
    print(f'clusters: {cxnn.cluster_count()}')
        
def learn_cycle(cxnn: ContextNN, 
                feeder: PhraseFeeder, 
                bit_notes: BitNotes,
                batch_size=200,
                n_accumulate_batches=30,
                n_consolidate_batches=50,
                reduce_min_component=0.2,
                reduce_min_activations=100):
    print('\naccumulating clusters...')
    cxnn.state = const.STATE_ACCUMULATE
    feed_n_batches(cxnn, feeder, bit_notes, n=n_accumulate_batches, batch_size=batch_size)
    
    print('\nconsolidating clusters...')
    cxnn.state = const.STATE_CONSOLIDATE
    feed_n_batches(cxnn, feeder, bit_notes, n=n_consolidate_batches, batch_size=batch_size)
    
    print('\nreducing clusters...')
    cxnn.reduce_clusters(min_component=reduce_min_component, 
                         min_activations=reduce_min_activations,
                         trim=True,
                         remain_parts=3,
                         clear_stats=True,
                         consolidate=True,
                         amnesty=True)
    print(f'clusters: {cxnn.cluster_count()}')
    

In [4]:
notes = bitnotes.BitNotes(note_count=26, 
                          notation_count=5, 
                          active_bits=8, 
                          bit_count=256)

feeder = phrase_feeder.PhraseFeeder(phrase_base, marks)  

cxnn = context_nn.ContextNN(input_bit_count=256,
                            output_bit_count=len(marks.keys()),
                            watch_point_count=20000,
                            watch_bit_count=32,
                            cluster_make_threshold=7,
                            cluster_activate_threshold=5,
                            bit_notes=notes,
                            data_marks=marks)

In [6]:
%%time
for n in range(5):
    print(f'\nCycle {n+1}')
    learn_cycle(cxnn, 
                feeder, 
                notes,
                batch_size=200,
                n_accumulate_batches=32,
                n_consolidate_batches=32,
                reduce_min_component=0.15,
                reduce_min_activations=100)


Cycle 1

accumulating clusters...
batch 1/32 bel
batch 2/32 blg
batch 3/32 eng
batch 4/32 epo
batch 5/32 jbo
batch 6/32 pol
batch 7/32 rus
batch 8/32 ukr
batch 9/32 bel
batch 10/32 blg
batch 11/32 eng
batch 12/32 epo
batch 13/32 jbo
batch 14/32 pol
batch 15/32 rus
batch 16/32 ukr
batch 17/32 bel
batch 18/32 blg
batch 19/32 eng
batch 20/32 epo
batch 21/32 jbo
batch 22/32 pol
batch 23/32 rus
batch 24/32 ukr
batch 25/32 bel
batch 26/32 blg
batch 27/32 eng
batch 28/32 epo
batch 29/32 jbo
batch 30/32 pol
batch 31/32 rus
batch 32/32 ukr
clusters: 1958160

consolidating clusters...
batch 1/32 bel
batch 2/32 blg
batch 3/32 eng
batch 4/32 epo
batch 5/32 jbo
batch 6/32 pol
batch 7/32 rus
batch 8/32 ukr
batch 9/32 bel
batch 10/32 blg
batch 11/32 eng
batch 12/32 epo
batch 13/32 jbo
batch 14/32 pol
batch 15/32 rus
batch 16/32 ukr
batch 17/32 bel
batch 18/32 blg
batch 19/32 eng
batch 20/32 epo
batch 21/32 jbo
batch 22/32 pol
batch 23/32 rus
batch 24/32 ukr
batch 25/32 bel
batch 26/32 blg
batch 27/3

### Model stats

In [10]:
print(cxnn.cluster_count())
print(sorted(cxnn.point_stats(), key=lambda x: x[0]))
# print(cxnn.point_stats())
marks

66439
[(0, 0), (0, 0), (0, 14), (0, 1), (0, 0), (0, 14), (0, 0), (0, 13), (0, 0), (0, 154), (0, 7), (0, 0), (0, 0), (0, 15), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 6), (0, 0), (0, 0), (0, 0), (0, 2), (0, 0), (0, 2), (0, 1), (0, 0), (0, 0), (0, 2), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 0), (0, 1), (0, 14), (0, 0), (0, 0), (0, 140), (0, 0), (0, 0), (0, 1), (0, 0), (0, 81), (0, 0), (0, 347), (0, 0), (0, 0), (0, 7), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 30), (0, 47), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 2), (0, 0), (0, 35), (0, 1), (0, 0), (0, 1), (0, 0), (0, 6), (0, 0), (0, 0), (0, 9), (0, 0), (0, 0), (0, 0), (0, 5), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 9), (0, 46), (0, 0), (0, 0), (0, 0), (0, 6), (0, 0), (0, 151), (0, 2), (0, 0), (0, 0), (0, 5), (0, 66), (0, 0), (0, 0), (0, 0), (0, 79), (0, 0), (0, 4), (0, 0), (0, 0), (0, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 186), (0, 0), (0, 0), (0, 2), (0, 71), (0

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [10]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

158335
{6: 1743, 7: 47851, 8: 101299, 9: 7057, 10: 343, 11: 39, 12: 3}
{0: 158335}


In [9]:
cxnn.cluster_consolidated_stats()

{0: 16848,
 1: 32286,
 2: 21541,
 3: 16593,
 4: 15634,
 5: 12351,
 6: 12131,
 7: 8745,
 8: 8752,
 9: 6707,
 10: 6747}

In [ ]:
def dict_value(d: dict, n=0) -> object:
    i = 0
    diter = iter(d)
    while i <= n:
        key = next(diter)
        i += 1
    return d[key]


point = cxnn.point_objects[8]
cluster_idx = 0
cluster = point.cluster_objects[cluster_idx]
pprint(cluster.stats)
print(sum(cluster.stats.values()))
# pprint(cluster.component_stats())
# pprint(cluster.bit_rate())

print(sorted(cluster.component_stats().items(), key=lambda x: x[1], reverse=True))
print(point.cluster_masks)
print(np.where(cluster.bit_mask > 0)[0])
print(point.watch_bits)
print(point.cluster_masks[cluster_idx])
print(len(point.cluster_masks), len(point.cluster_objects))

# parts = cluster.component_s
# enthropy = 0.0
# for part in parts.values():
#     enthropy += -part * math.log2(part)
# print(enthropy)

In [ ]:
cluster_with_stats = None
for wp in cxnn.point_objects:
    for cluster in wp.cluster_objects:    
        if cluster.stats:
            cluster_with_stats = cluster
            break
    else: 
        continue
    break    

if cluster_with_stats:
    print(cluster_with_stats.stats, cluster_with_stats.consolidated)
else:
    print('nothing to print')
    
    

### Save-load model

In [7]:
import pickle

def save_cxnn(cxnn: ContextNN, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(cxnn, f)

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def save_notes(notes: BitNotes, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(notes, f)

def load_notes(file_name: str) -> BitNotes:
    with open(file_name, 'rb') as f:
        notes = pickle.load(f)
        return notes



In [8]:
save_cxnn(cxnn, './data/cxnn_langs.pickle')

In [ ]:
cxnn = load_cxnn('./data/cxnn_langs.pickle')

In [ ]:
save_cxnn(cxnn, './data/cxnn_monotone.pickle')